In [1]:
"""Implementations of algorithms for continuous control."""
import functools
from jaxrl_m.typing import *

import jax
import jax.lax as lax
import jax.numpy as jnp
import numpy as np
import optax
from jaxrl_m.common import TrainState, target_update, nonpytree_field
from jaxrl_m.networks import DeterministicPolicy,Policy, Critic, ensemblize

import flax
import flax.linen as nn

from functools import partial



class SACAgent(flax.struct.PyTreeNode):
    rng: PRNGKey
    critic: (TrainState)
    actor: TrainState
    config: dict = nonpytree_field()
    
    
        
    

    @partial(jax.jit,static_argnames=('num_steps',))  
    
    
    def update_many_critics(agent,transitions: Batch,idxs:jnp.array,num_steps:int,R2):
        
        
        
        def update_one_critic(critic_params,critic_opt_state,batch_stats,idxs,
                            agent,transitions,num_steps):
            
            
            def one_update(agent,
                        critic_params,critic_opt_state,batch_stats,
                        batch: Batch):
                                  
                def critic_loss_fn(critic_params):
                    next_actions = agent.actor(batch['next_observations'])
                    concat_actions = jnp.concatenate([batch["actions"],next_actions])
                    concat_observations = jnp.concatenate([batch["observations"],batch["next_observations"]])
                    
                    concat_q,updates = agent.critic(concat_observations, concat_actions,True,
                                                    params=critic_params,extra_variables=batch_stats,mutable=["batch_stats"])
                    q,next_q = jnp.split(concat_q,2,axis=0) ## axis=1 for ensemble
                    target_q = batch['rewards'] + agent.config['discount'] * batch['masks'] * next_q
                    target_q = jax.lax.stop_gradient(target_q)
                    
                    critic_loss = ((target_q-q)**2).mean()
                    
                    return critic_loss, {"batch_stats":updates["batch_stats"]}
        

                grads,batch_stats = jax.grad(critic_loss_fn, has_aux=True)(critic_params)
                updates, new_opt_state = agent.critic.tx.update(grads, critic_opt_state, critic_params)
                new_params = optax.apply_updates(critic_params, updates)

                
                return agent,new_params,new_opt_state,batch_stats
            
            
            get_batch = lambda transitions,idx : jax.tree_map(lambda x : x[idx],transitions)
            
            
            agent,critic_params,critic_opt_state,batch_stats = jax.lax.fori_loop(0, num_steps, 
                        lambda i, args: one_update(*args,get_batch(transitions,idxs[i])),
                        (agent,critic_params,critic_opt_state,batch_stats))
            
            
            
            # args = (agent,critic_params,critic_opt_state,batch_stats)
            # agent,critic_params,critic_opt_state,batch_stats =   one_update(*args,get_batch(transitions,idxs[0]))
            
                        
            
            return critic_params,critic_opt_state,batch_stats
        
        
        new_rng, curr_key, next_key = jax.random.split(agent.rng, 3)
        
        critic = agent.critic
        ###### Reset optimizer params ######
        opt_state = jax.vmap(critic.tx.init,in_axes=0)(agent.critic.params)
        ###### Reset critic params ######
        
        reset = lambda rng,params : critic.init(rng,
                                                agent.config["observations"], agent.config["actions"],False)["params"]
        no_reset = lambda rng,params: params
        f = lambda  mask,rng,params :lax.cond(mask,reset,no_reset,rng,params)
        mask = jnp.zeros((10))
        mask.at[jnp.argmin(R2)].set(1)
        rngs = jax.random.split(agent.rng, 10)
        critic_params = jax.vmap(f,in_axes=(0,0,0))(mask,rngs,critic.params)
        ###################################
        tmp = partial(update_one_critic,agent=agent,transitions=transitions,num_steps=num_steps)
        batch_stats = critic.extra_variables
        critic_params,critic_opt_state,batch_stats = jax.vmap(tmp,in_axes=(0,0,0,0))(critic_params,opt_state,batch_stats,idxs)
        new_critic = critic.replace(params=critic_params,opt_state=critic_opt_state,extra_variables=batch_stats)
        agent = agent.replace(rng=new_rng,critic=new_critic)
        
    
        return agent,{}
       
    

        
    @jax.jit
    def update_actor(agent, batch: Batch,q_weights,sample_weights):
        new_rng, curr_key, next_key = jax.random.split(agent.rng, 3)

        def actor_loss_fn(actor_params,actor_stats,q_weights):
            
            actions,updates = agent.actor(batch['observations'],True, params=actor_params,extra_variables=actor_stats,mutable=["batch_stats"])
            
            call_one_critic = lambda observations,actions,params,batch_stats: agent.critic(observations,actions,
                                                                                           params=params,extra_variables=batch_stats)
            q = jax.vmap(call_one_critic,in_axes=(None,None,0,0))(batch['observations'], actions,agent.critic.params,agent.critic.extra_variables)
            q_weights = jax.nn.softmax(q_weights,axis=0)
            q = jnp.sum(q_weights*q,axis=0)
            q = q*sample_weights
            
            actor_loss = (-q).mean()
            
            return actor_loss, updates["batch_stats"]

        loss_fn = partial(actor_loss_fn,actor_stats=agent.actor.extra_variables,q_weights=q_weights)
        new_actor, batch_stats = agent.actor.apply_loss_fn(loss_fn=loss_fn, has_aux=True)
        new_actor = new_actor.replace(extra_variables={"batch_stats":batch_stats})
        actor_info = {}
        return agent.replace(rng=new_rng,actor=new_actor,), {**actor_info}
        


    @jax.jit
    def sample_actions(agent,observations: np.ndarray) -> jnp.ndarray:
        actions = agent.actor(observations)
       
        return actions
    
 

def create_learner(
                 seed: int,
                 observations: jnp.ndarray,
                 actions: jnp.ndarray,
                 actor_lr: float = 3e-4,
                 critic_lr: float = 3e-4,
                 hidden_dims: Sequence[int] = (256, 256),
                 discount: float = 0.99,
                 tau: float = 0.005,
            **kwargs):

        print('Extra kwargs:', kwargs)

        rng = jax.random.PRNGKey(seed)
        rng, actor_key, critic_key = jax.random.split(rng, 3)

        action_dim = actions.shape[-1]
        actor_def = DeterministicPolicy((64,64), action_dim=action_dim,final_fc_init_scale=1.0)

        actor_tmp = actor_def.init(actor_key, observations,training=True)
        actor_params = actor_tmp['params']
        actor_stats = actor_tmp['batch_stats']
        actor = TrainState.create(actor_def, actor_params, tx=optax.adam(learning_rate=actor_lr),
                                  extra_variables={"batch_stats":actor_stats})
        
        critic_def = Critic(hidden_dims)
        critic_tmp = critic_def.init(critic_key, observations, actions,False)
        critic_params = critic_tmp['params']
        critic_stats = critic_tmp['batch_stats']
        tmp_critic = TrainState.create(critic_def, critic_params,optax.adam(learning_rate=critic_lr),
                                       extra_variables={"batch_stats":critic_stats})
        
        critic_keys  = jax.random.split(critic_key, 10)
        critic_tmp = jax.vmap(critic_def.init,in_axes=(0,None,None))(critic_keys, observations, actions)
        critic_params = critic_tmp['params']
        critic_stats = critic_tmp['batch_stats']
        critic = tmp_critic.replace(params=critic_params,
                                    extra_variables={"batch_stats":critic_stats}) ## opt_state is updated in "update_many_critics"
        
        
        print("Initialisation",jax.tree_map(lambda x : x.shape,critic_stats))

        
        config = flax.core.FrozenDict(dict(
            discount=discount,
            target_update_rate=tau, 
            observations = observations,
            actions = actions,
        ))

        return SACAgent(rng, critic=critic, actor=actor, config=config)

In [2]:
import functools

def compute_q(anc_agent,obs,actor_params,actor_stats,
              critic_params,critic_stats):

    actions = anc_agent.actor(obs, params=actor_params,extra_variables=actor_stats)
    q = anc_agent.critic(obs, actions,False,params=critic_params,extra_variables=critic_stats)
   
    return q
    
    

def estimate_return(acq_rollout,
                    anc_agent,anc_critic_params,anc_critic_stats,anc_return):
    
    acq_obs = acq_rollout.observations
    acq_masks = acq_rollout.disc_masks
    acq_return = acq_rollout.policy_return
    
    anc_actor_params = anc_agent.actor.params
    anc_actor_stats = anc_agent.actor.extra_variables
    acq_actor_params = acq_rollout.policy_params
    acq_actor_stats = acq_rollout.policy_stats
      
    anc_q = compute_q(anc_agent,acq_obs,
                      anc_actor_params,anc_actor_stats,
                      anc_critic_params,anc_critic_stats)
    acq_q = compute_q(anc_agent,acq_obs,
                      acq_actor_params,acq_actor_stats,
                      anc_critic_params,anc_critic_stats)
    
    adv = ((acq_q - anc_q)*acq_masks).sum()/5
    acq_return_pred = anc_return + adv
  
    
    return acq_return_pred,acq_return


def evaluate_critic(anc_agent,anc_critic_params,anc_critic_stats,
                    anc_return,policy_rollouts):

    
    tmp =  partial(estimate_return,
                   anc_agent=anc_agent,
                   anc_critic_params =anc_critic_params,
                   anc_critic_stats =anc_critic_stats,
                   anc_return = anc_return)
    y_pred,y = jax.vmap(tmp)(policy_rollouts)
    a2 = jnp.clip(((y-y_pred)**2),a_min=1e-4).sum()
    b2=((y-y.mean())**2).sum()
    R2 = 1-(a2/b2)  
    bias = (y_pred-y).mean()
    
    return R2,bias

@jax.jit
def evaluate_critics(anc_agent,anc_critic_params,anc_critic_stats,
                    anc_return,policy_rollouts):
    
    return jax.vmap(evaluate_critic,in_axes=(None,0,0,None,None))(anc_agent,anc_critic_params,anc_critic_stats,
                                                                  anc_return,policy_rollouts)



def merge(x,y):

    return jax.tree_map(lambda x,y : jnp.vstack([x,y]),x,y)

def flatten_rollouts(policy_rollouts):
    
    n_policies = len(policy_rollouts)
    merged_rollouts = functools.reduce(merge, policy_rollouts)
    merged_rollouts = jax.tree_map(lambda x:jnp.stack(jnp.split(x,n_policies,axis=0)),merged_rollouts)
    
    def reshape_tree(tree, reference_tree,n_policies):
        def reshape_fn(x, reference_x):
            return jnp.reshape(x, (n_policies,*reference_x.shape))
        
        return jax.tree_map(reshape_fn, tree, reference_tree)
    
    merged_rollouts = reshape_tree(merged_rollouts,policy_rollouts[0],n_policies)
    
    return merged_rollouts


In [3]:
import os
from functools import partial
import numpy as np
import jax
import tqdm
import gymnasium as gym

from jaxrl_m.common import CodeTimer
from jaxrl_m.wandb import setup_wandb, default_wandb_config, get_flag_dict
import wandb
from jaxrl_m.evaluation import supply_rng, evaluate, flatten, EpisodeMonitor
from jaxrl_m.dataset import ReplayBuffer
from collections import deque
from jaxrl_m.rollout import PolicyRollout,rollout_policy

env_name='InvertedPendulum-v4'
seed=np.random.choice(1000000)
eval_episodes=10
batch_size = 256
max_steps = int(1e6)
start_steps = 5000                   
log_interval = 5000
#eval_interval = 10000

wandb_config = default_wandb_config()
wandb_config.update({
    'project': 'd4rl_test',
    'group': 'sac_test',
    'name': 'sac_{env_name}',
})


env = EpisodeMonitor(gym.make(env_name))
eval_env = EpisodeMonitor(gym.make(env_name))
setup_wandb({"bonjour":1})

example_transition = dict(
    observations=env.observation_space.sample(),
    actions=env.action_space.sample(),
    rewards=0.0,
    masks=1.0,
    next_observations=env.observation_space.sample(),
    discounts=1.0,
)

replay_buffer = ReplayBuffer.create(example_transition, size=int(1e6))
actor_buffer = ReplayBuffer.create(example_transition, size=int(5e3))

agent = create_learner(seed,
                example_transition['observations'][None],
                example_transition['actions'][None],
                max_steps=max_steps,
                #**FLAGS.config
                )

exploration_metrics = dict()
obs,info = env.reset()    
exploration_rng = jax.random.PRNGKey(0)
i = 0
num_grad_updates = 0
unlogged_steps = 0
policy_rollouts = deque([], maxlen=20)
R2 = jnp.ones(10)
with tqdm.tqdm(total=max_steps) as pbar:
    
    while i < max_steps:


        #with CodeTimer('rollout'):
            
            replay_buffer,actor_buffer,policy_rollout,policy_return,undisc_policy_return,num_steps = rollout_policy(agent,env,exploration_rng,
                    replay_buffer,actor_buffer,
                    warmup=(i < start_steps))
            policy_rollouts.append(policy_rollout)
            unlogged_steps += num_steps
            i+=num_steps
            pbar.update(num_steps)
            
            
        
            if replay_buffer.size > start_steps:
        
        
            #with CodeTimer('update_critic'):
                key = jax.random.PRNGKey(0)
                transitions = replay_buffer.get_all()
                tmp = partial(jax.random.choice,a=replay_buffer.size, shape=(10000,256), replace=True)
                idxs = jax.vmap(tmp)(jax.random.split(key, 10))
                agent, critic_update_info = agent.update_many_critics(transitions,idxs,10000,R2)
            
    #        with CodeTimer('evaluate_critic'):        
                critic_params = agent.critic.params
                critic_stats = agent.critic.extra_variables
                anc_return = policy_rollouts[-1].policy_return
                flattened_rollouts = flatten_rollouts(policy_rollouts)
                R2,bias = evaluate_critics(agent,critic_params,critic_stats,
                                           anc_return,flattened_rollouts)

            #with CodeTimer('update_actor'):
                actor_batch = actor_buffer.get_all() 
                sample_weights = policy_rollouts[-1].disc_masks
                agent, actor_update_info = agent.update_actor(actor_batch,R2.reshape(-1,1),sample_weights)   
                num_grad_updates += 1 
                
            
                
                if unlogged_steps > log_interval:
                    
                    #with CodeTimer('eval'):
                            
                        
                        update_info = {**critic_update_info, **actor_update_info,
                                'R2_validation': jnp.max(R2),'bias': jnp.max(bias),'num_grad_updates': num_grad_updates
                                }
                        exploration_metrics = {f'exploration/disc_return': policy_return}
                        
                        train_metrics = {f'training/{k}': v for k, v in update_info.items()}
                        
                        # eval_info = evaluate(agent.actor, eval_env, num_episodes=eval_episodes)
                        # eval_metrics = {f'evaluation/{k}': v for k, v in eval_info.items()}
                        
                    #with CodeTimer('logging'):
                        
                        wandb.log(train_metrics, step=int(i),commit=False)
                        wandb.log({"undisc_return":undisc_policy_return},step=int(i))
                        wandb.log(exploration_metrics, step=int(i),commit=True)
                        #wandb.log(eval_metrics, step=int(i),commit=True)
                        unlogged_steps = 0
                    

2023-12-25 14:29:11.201557: W external/xla/xla/service/gpu/nvptx_compiler.cc:679] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.3.103). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin


Extra kwargs: {'max_steps': 1000000}
Initialisation {'MLP_0': {'BatchNorm_0': {'mean': (10, 256), 'var': (10, 256)}, 'BatchNorm_1': {'mean': (10, 256), 'var': (10, 256)}}}


 29%|██▉       | 294999/1000000 [10:09<24:16, 483.91it/s] 


KeyboardInterrupt: 